In [ ]:
def cleanup_scene():
    """Removes existing piecelets and rotation empties directly from data."""
    # No need to select/deselect for this method
    objs_to_remove = [o for o in bpy.data.objects if o.name.startswith("Piece.") or o.name.startswith("ROT_")]

    if objs_to_remove:
        print(f"Removing {len(objs_to_remove)} old objects...")
        for obj in objs_to_remove:
            bpy.data.objects.remove(obj, do_unlink=True) # Use direct removal
        print("Finished removing old objects.")
    else:
        print("No old objects found to remove.")

    # Optional: Clear animation data (use with caution if scene has other animations)
    # for action in bpy.data.actions:
    #     bpy.data.actions.remove(action)

    # Ensure the viewport updates if anything was removed
    bpy.context.view_layer.update()

In [5]:
from collections import deque
f = deque(['a','b','c','d'])
f.appendleft(0)
f[0]
[i for i in enumerate(f)]

[(0, 0), (1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]

In [15]:
import numpy as np

aa = np.array([
        [[0 , 1 , 2 ],
         [3 , 4 , 5 ],
         [6 , 7 , 8 ]], # Front face

        [[9 , 10, 11],
         [12, 13, 14],
         [15, 16, 17]], # Middle slice

        [[18, 19, 20],
         [21, 22, 23],
         [24, 25, 26]], # Back face
])

cube_faces = {
    'F': aa[0, :, :],
    'B': np.flip(aa[2, :, :], axis=1),
    'R': aa[:, :, 2],
    'L': np.flip(np.transpose(aa[:, :, 0]), axis=1),
    'U' : np.flip(aa[:, 0, :], axis=0),
    'D' : aa[:, 2, :]
}
for i in cube_faces:
    print(i, cube_faces[i])

F [[0 1 2]
 [3 4 5]
 [6 7 8]]
B [[20 19 18]
 [23 22 21]
 [26 25 24]]
R [[ 2  5  8]
 [11 14 17]
 [20 23 26]]
L [[18  9  0]
 [21 12  3]
 [24 15  6]]
U [[18 19 20]
 [ 9 10 11]
 [ 0  1  2]]
D [[ 6  7  8]
 [15 16 17]
 [24 25 26]]


In [13]:
np.flip(np.transpose(aa[:, :, 0]), axis=1)

array([[18,  9,  0],
       [21, 12,  3],
       [24, 15,  6]])

In [17]:
a = [1,2]
a.reverse()
a

[2, 1]

In [25]:
import numpy as np
m = np.array([[1,2],[3,4]], int)
m = np.rot90(m, k=0, axes=(0, 1))
m

array([[1, 2],
       [3, 4]])